In [8]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [7]:
data = pd.read_csv("covid_toy.csv")
data.head()

,Unnamed: 0,age,gender,fever,cough,city,has_covid
0,0,60,Male,103.0,Mild,Kolkata,No
1,1,27,Male,100.0,Mild,Delhi,Yes
2,2,42,Male,101.0,Mild,Delhi,No
3,3,31,Female,98.0,Mild,Kolkata,No
4,4,65,Female,101.0,Mild,Mumbai,No


In [10]:
data['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['has_covid']), data['has_covid'], test_size=0.2)


1. without column transformer

In [21]:
si = SimpleImputer()
# for training data
X_train_fever = si.fit_transform(X_train[['fever']])

# for testing data
X_test_fever = si.transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [23]:
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# same for the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

In [27]:
ohe = OneHotEncoder(sparse=False, drop='first')

X_train_gender_city = ohe.fit_transform(X_train[['gender', 'city']])

#same for the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender', 'city']])


In [30]:
#extracting age

X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

In [31]:
X_train_transform = np.concatenate((X_train_age, X_train_fever, X_train_gender_city, X_train_cough), axis=1)

In [32]:
X_train_transform

array([[ 27.        ,  33.        , 102.        ,   0.        ,
          1.        ,   0.        ,   0.        ,   1.        ],
       [ 96.        ,  51.        , 101.        ,   0.        ,
          0.        ,   1.        ,   0.        ,   1.        ],
       [ 65.        ,  69.        , 102.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [ 66.        ,  51.        , 104.        ,   1.        ,
          0.        ,   1.        ,   0.        ,   0.        ],
       [ 37.        ,  55.        , 100.        ,   1.        ,
          0.        ,   1.        ,   0.        ,   0.        ],
       [ 10.        ,  75.        , 100.73611111,   0.        ,
          1.        ,   0.        ,   0.        ,   0.        ],
       [  2.        ,  42.        , 101.        ,   1.        ,
          1.        ,   0.        ,   0.        ,   0.        ],
       [ 58.        ,  23.        ,  98.        ,   1.        ,
          0.        ,   0.       

2 transforming all the columns using column transformer

In [33]:
from sklearn.compose import ColumnTransformer

In [39]:
transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(sparse=False, drop='first'), ['gender', 'city'])
], remainder='passthrough')

In [46]:
transformer.fit_transform(X_train)


array([[101.,   0.,   0.,   1.,   0.,   0.,   9.,  64.],
       [104.,   0.,   1.,   0.,   0.,   1.,  49.,  44.],
       [ 99.,   1.,   0.,   0.,   0.,   0.,  57.,  49.],
       [102.,   1.,   0.,   0.,   0.,   0.,  60.,  24.],
       [103.,   1.,   1.,   0.,   0.,   0.,  89.,  46.],
       [100.,   0.,   1.,   0.,   0.,   0.,  78.,  11.],
       [103.,   0.,   1.,   0.,   1.,   0.,   0.,  60.],
       [101.,   0.,   0.,   0.,   0.,   1.,  46.,  19.],
       [ 98.,   0.,   0.,   1.,   0.,   0.,  23.,  80.],
       [101.,   1.,   1.,   0.,   1.,   0.,  56.,  71.],
       [103.,   0.,   0.,   0.,   1.,   0.,  39.,  50.],
       [101.,   0.,   0.,   0.,   0.,   1.,   4.,  65.],
       [101.,   1.,   0.,   0.,   0.,   1.,  29.,  34.],
       [102.,   1.,   1.,   1.,   0.,   0.,  44.,  20.],
       [ 98.,   1.,   1.,   0.,   0.,   0.,  20.,  12.],
       [101.,   0.,   0.,   0.,   0.,   0.,  97.,  20.],
       [100.,   0.,   1.,   0.,   0.,   0.,  76.,  80.],
       [ 99.,   0.,   1.,   1.,